# Detector de correos Spam

### Integrantes
- Beltran Medina Carlos Daniel
- Beltran Ontiveros Karen Valeria

### 1. Importar las librerias necesarias

In [317]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from nltk.corpus import stopwords

### 2. Cargar los correos del dataset

In [318]:
correos = pd.read_csv("dataset/emails.csv")

### 3. Preprocesamiento de los datos

In [319]:
# 1. Eliminar registros duplicados
correos = correos.drop_duplicates(subset="text").copy()

# 3. Convertir el texto a minúsculas
correos['text'] = correos['text'].str.lower()

# 4. Obtener stopwords en español e inglés
palabras_vacias = set(stopwords.words('spanish')).union(set(stopwords.words('english')))

# 5. Eliminar palabras vacías (stopwords)
correos['text'] = correos['text'].apply(lambda x: ' '.join(
    palabra for palabra in x.split() if palabra not in palabras_vacias))

# 6. Filtrar correos que estén vacíos
correos = correos[correos['text'].str.strip() != '']

# 7. Tokenizar el texto
correos['text'] = correos['text'].apply(lambda x: x.split())

# 8. Agrupar los correos por la columna 'target' (spam/no spam)
correos.groupby('spam').describe()

text                                                               
     count unique                                                top freq
spam                                                                     
0     4327   4165  [subject:, california, update, 4, /, 27, /, 01...    2
1     1368   1367  [subject:, charity, sees, need, cost, ., ., .,...    2

### 4. Modelo

In [320]:
# Vectorizar los correos electrónicos
vectorizador = TfidfVectorizer(stop_words="english")
caracteristicas = vectorizador.fit_transform(correos['text'].apply(lambda x: ' '.join(x)))

# Calcular la probabilidad previa de spam (P(spam))
p_spam = correos['spam'].mean()

# Calcular la probabilidad previa de no spam (P(no spam))
p_no_spam = 1 - p_spam 

# Calcular la probabilidad de las características dado spam (P(caracteristicas|spam))
mask_spam = correos['spam'].to_numpy().astype(int) == 1
suma_spam = caracteristicas[mask_spam].sum(axis=0)
p_caracteristicas_spam = np.array(suma_spam / suma_spam.sum()).flatten()

# Calcular la probabilidad de las características dado no spam (P(caracteristicas|no spam))
mask_no_spam = correos['spam'].to_numpy().astype(int) == 0
suma_no_spam = caracteristicas[mask_no_spam].sum(axis=0)
p_caracteristicas_no_spam = np.array(suma_no_spam / suma_no_spam.sum()).flatten()

# Calcular la probabilidad posterior de que el correo sea spam (P(spam|caracteristicas))
p_spam_por_correo = caracteristicas.dot(p_caracteristicas_spam.T)
p_no_spam_por_correo = caracteristicas.dot(p_caracteristicas_no_spam.T)
p_spam_caracteristicas = (p_spam * p_spam_por_correo) / (
    (p_spam * p_spam_por_correo) + (p_no_spam * p_no_spam_por_correo)
)


### 5. Clasificacion

In [321]:
clasificaciones = np.where(p_spam_caracteristicas > 0.5, 1, 0)

# Convertir los valores reales de spam a un array
spam_labels = correos['spam'].to_numpy().astype(int)

### 6. Evaluacion

In [322]:
# Calcular la precisión
precision = np.sum(clasificaciones == spam_labels) / len(clasificaciones)
print(f"Precisión del clasificador: {precision:.2f}")
recuperacion = np.sum((clasificaciones == 1) & (spam_labels == 1)) / np.sum(spam_labels == 1)
print(f"Recuperación del clasificador: {recuperacion:.2f}")

Precisión del clasificador: 0.83
Recuperación del clasificador: 0.30


### 7. Probar con un correo

In [323]:
# Asegurar que p_caracteristicas_spam y p_caracteristicas_no_spam sean vectores columna
p_caracteristicas_spam = p_caracteristicas_spam.reshape(-1, 1)
p_caracteristicas_no_spam = p_caracteristicas_no_spam.reshape(-1, 1)

In [324]:
#correo = "Hello Manuel, I am writing to you to invite you to the party in Cesar's house"
correo = "You have won a prize, click here to claim it"

# Vectorizar el correo nuevo
correo_vectorizado = vectorizador.transform([correo])

# Calcular P(caracteristicas|spam) y P(caracteristicas|no spam)
p_spam_correo = (correo_vectorizado @ p_caracteristicas_spam)[0, 0]
p_no_spam_correo = (correo_vectorizado @ p_caracteristicas_no_spam)[0, 0]

# Evitar división por cero
denominador = (p_spam * p_spam_correo) + (p_no_spam * p_no_spam_correo)
if denominador == 0:
    p_spam_correo_final = 0
else:
    p_spam_correo_final = (p_spam * p_spam_correo) / denominador

# Mostrar resultados
if p_spam_correo_final > 0.5:
    print("El correo es spam.")
else:
    print("El correo no es spam.")

El correo es spam.


### Extra: Utilizando libreria para el teorema de bayes

In [325]:
# 1. Inicializar el vectorizador
vectorizador = CountVectorizer()

# 2. Dividir los datos en conjuntos de entrenamiento y prueba
entrenamiento_x, prueba_x, entrenamiento_y, prueba_y = train_test_split(
    correos['text'].apply(lambda x: ' '.join(x)),
    correos['spam'],
    test_size=0.25,
    random_state=42
)

# 3. Ajustar y transformar los datos de entrenamiento
matriz_entrenamiento_x = vectorizador.fit_transform(entrenamiento_x)

# 4. Inicializar el modelo Naive Bayes Multinomial
modelo_bayes = MultinomialNB()

# 5. Entrenar el modelo
modelo_bayes.fit(matriz_entrenamiento_x, entrenamiento_y)

# 6. Preparar los datos de prueba
matriz_prueba_x = vectorizador.transform(prueba_x)

# 7. Evaluar el modelo con los datos de prueba
precision = modelo_bayes.score(matriz_prueba_x, prueba_y)
print(f"\nPrecisión del modelo: {precision}")

# 8. Predecir el correo nuevo
#correo = "Hello Manuel, I am writing to you to invite you to the party in Cesar's house"
correo = "You have won a prize, click here to claim it"
correoVectorizado = vectorizador.transform([correo])
prediccion = modelo_bayes.predict(correoVectorizado)
print(f"\nEl correo ingresado es {'spam' if prediccion == 1 else 'no spam'}")


Precisión del modelo: 0.9908707865168539

El correo ingresado es spam
